### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.automation :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.indicators :refer :all]
            [clojure-backtesting.direct :refer :all]
            [clojure.string :as str]
            [clojure.java.io :as io]
            [clojure.pprint :as pprint]
  ) ;; require all libriaries from core
)

nil

### Import dataset

In [2]:
; path to dataset = "/Volumes/T7/CRSP"
; change it to the relative path to your own dataset
;
(load-dataset "/Volumes/T7/CRSP" "main" add-aprc)

### Initialise portfolio （Go back here everytime you want to restart.）

In [3]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 1000);

### Relative Strength Index (RSI) strategy

<p>RSI = 100 - 100 / (1 + RS)</p>
RS = Average Gain / Average Loss

<p>1st Average Gain = Sum of gains over past 14 periods / 14</p>
1st Average Loss = Sum of losses over past 14 periods / 14

<p>Average Gain = (Previous average gain x 13 + Current gain) / 14</p>
Average Loss = (Previous average loss x 13 + Current loss) / 14

We'll trade according to the following rules:
- **Buy signal**: when RSI crosses the lower threshold (e.g. 30)
- **Sell signal**: when RSI crosses the upper threshold (e.g. 70)

In [6]:
(get-date)

"1980-12-15"

In [7]:
(def n 14)

;; go to 14 days after
(for [x (range n)] 
  (next-date))

(get-date)

"1981-01-06"

In [11]:
;; calculate average gains and losses for remaining days
(while (< (compare (get-date) "1981-03-31") 0)
  (do
    (let [rsi (RSI "14593")]
      (if (< rsi lower-threshold)
        (order "14593" 1 :print false)) ; buy signal
      (if (> rs upper-threshold)
        (order "14593" -1 :print false)) ; sell signal
      (update-eval-report)
      (next-date)))
(end-order))

[]

In [13]:
(print-order-record 10)


|      :date | :tic |  :price | :aprc | :quantity |
|------------+------+---------+-------+-----------|
| 1981-01-08 | AAPL | 30.3125 | 27.92 |         1 |
| 1981-01-09 | AAPL | 31.9375 | 28.56 |         1 |
| 1981-01-12 | AAPL |   31.75 | 28.49 |         1 |
| 1981-01-13 | AAPL | 30.5625 | 28.02 |         1 |
| 1981-01-14 | AAPL | 30.6875 | 28.07 |         1 |
| 1981-01-15 | AAPL |  31.375 | 28.34 |         1 |
| 1981-01-16 | AAPL | 31.0625 | 28.22 |         1 |
| 1981-01-19 | AAPL | 32.9375 | 28.95 |         1 |
| 1981-01-20 | AAPL | 31.9375 | 28.56 |         1 |
| 1981-01-21 | AAPL |  32.625 | 28.83 |         1 |


nil

### Check portfolio record

In [14]:
;; view final portfolio
(print-portfolio)


| :asset | :price | :aprc | :quantity | :tot-val |
|--------+--------+-------+-----------+----------|
|   cash |    N/A |   N/A |       N/A |   891.05 |


nil

In [132]:
;; view portfolio value and return
(print-portfolio-record 10)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage | :margin |
|------------+------------+------------+----------+-------+-----------+---------|
| 1980-12-15 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1981-01-08 |   $1000.00 |      0.00% |    0.00% | $0.00 |      0.00 |   0.00% |
| 1981-01-09 |   $1000.64 |      0.00% |    0.03% | $0.00 |      0.00 |   0.00% |
| 1981-01-12 |   $1000.49 |      0.00% |    0.02% | $0.00 |      0.00 |   0.00% |
| 1981-01-13 |    $999.09 |      0.00% |   -0.04% | $0.00 |      0.00 |   0.00% |
| 1981-01-14 |    $999.29 |      0.00% |   -0.03% | $0.00 |      0.00 |   0.00% |
| 1981-01-15 |   $1000.65 |      0.00% |    0.03% | $0.00 |      0.00 |   0.00% |
| 1981-01-16 |    $999.91 |      0.00% |   -0.00% | $0.00 |      0.00 |   0.00% |
| 1981-01-19 |   $1005.00 |      0.00% |    0.22% | $0.00 |      0.00 |   0.00% |
| 1981-01-20 |   $1001.92 |      0.00% |    0.08% | $0.00 |      0.00 |   0.00% |


nil

### Generate evaluation report

In [16]:
(print-eval-report 30)


|      :date | :tot-value |    :vol |  :r-vol |    :sharpe |  :r-sharpe | :pnl-pt | :max-drawdown |
|------------+------------+---------+---------+------------+------------+---------+---------------|
| 1981-01-07 |      $1000 | 0.0000% | 0.0000% |    0.0000% |    0.0000% |      $0 |        0.0000 |
| 1981-01-08 |      $1000 | 0.0161% | 0.0161% |    1.7321% |    1.7321% |      $0 |      100.0000 |
| 1981-01-09 |      $1000 | 0.0032% | 0.0032% |    6.7821% |    6.7821% |      $0 |        0.0000 |
| 1981-01-12 |       $999 | 0.0273% | 0.0273% |   -1.4483% |   -1.4483% |      $0 |        0.0000 |
| 1981-01-13 |       $999 | 0.0035% | 0.0035% |   -8.7393% |   -8.7393% |      $0 |      100.0000 |
| 1981-01-14 |      $1000 | 0.0223% | 0.0223% |    1.2614% |    1.2614% |      $0 |      100.0000 |
| 1981-01-15 |       $999 | 0.0113% | 0.0113% |   -0.3470% |   -0.3470% |      $0 |        0.0000 |
| 1981-01-16 |      $1005 | 0.0735% | 0.0735% |    2.9466% |    2.9466% |      $0 |      100.0000 |

nil

### Plot variables

In [17]:
(def data (deref portfolio-value))

#'clojure-backtesting.demo/data

In [18]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

#'clojure-backtesting.demo/data-to-plot

In [19]:
(first data-to-plot)

{:date "1980-12-15", :tot-value 1000, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0, :margin 0.0, :plot "portfolio"}

In [21]:
(plot data-to-plot :plot :date :daily-ret false)